1. Preparación de los Datos
Etiquetado de las Imágenes: Para entrenar un modelo YOLOX, necesitas un conjunto de datos etiquetado con las cajas delimitadoras (bounding boxes) que indican la región de interés en cada imagen. Si ya tienes un dataset con estas anotaciones en formato COCO o similar, puedes saltarte este paso. Si no, deberás crear estas anotaciones.

Formato COCO: YOLOX utiliza el formato COCO para las anotaciones. Debes tener un archivo JSON que contenga las coordenadas de las cajas delimitadoras para cada imagen.

Si no tienes anotaciones, podrías considerar crear un pequeño dataset anotado manualmente o utilizar herramientas como LabelImg para etiquetar las imágenes.

2. Configuración del Entorno para YOLOX
Instalación de YOLOX: Si no lo has hecho, primero clona el repositorio de YOLOX y instala las dependencias:


Descarga de Pesos Preentrenados (opcional): Puedes descargar pesos preentrenados de YOLOX para iniciar tu entrenamiento con un modelo ya preentrenado en COCO, lo que puede acelerar el proceso y mejorar los resultados.

3. Entrenamiento del Modelo YOLOX-nano
Archivo de Configuración: YOLOX requiere un archivo de configuración para definir el tamaño del modelo, el tamaño de las imágenes, la tasa de aprendizaje, etc. Si usas YOLOX-nano, podrías comenzar con el archivo de configuración de YOLOX-nano que se encuentra en exps/default/yolox_nano.py.

Si necesitas cambiar algún parámetro, puedes copiar y editar este archivo.
Monitoreo del Entrenamiento: Puedes utilizar TensorBoard para monitorear el progreso del entrenamiento:


tensorboard --logdir YOLOX/YOLOX_outputs
4. Inferencia y Extracción de ROI
Uso del Modelo Entrenado: Una vez que el modelo esté entrenado, puedes usarlo para hacer inferencia en nuevas imágenes, obteniendo las coordenadas de las regiones de interés.


5. Aplicar el Modelo para Extracción de ROI
Una vez que el modelo esté entrenado y hayas realizado inferencias, puedes extraer las ROIs de las imágenes utilizando las coordenadas de las cajas delimitadoras obtenidas del modelo.


In [3]:
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
%cd YOLOX

!pip install -v -e .
!pip install torch torchvision cython
!pip install -U opencv-python-headless
!pip install -U albumentations
!pip install -U tensorboard


Cloning into 'YOLOX'...
remote: Enumerating objects: 1925, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 1925 (delta 1), reused 11 (delta 0), pack-reused 1908 (from 1)
Receiving objects: 100% (1925/1925), 7.55 MiB | 3.72 MiB/s, done.
Resolving deltas: 100% (1147/1147), done.
on a case-insensitive filesystem) and only one from the same
colliding group is in the working tree:

  'demo/ncnn/android/app/src/main/java/com/megvii/yoloXncnn/YOLOXncnn.java'
  'demo/ncnn/android/app/src/main/java/com/megvii/yoloXncnn/yoloXncnn.java'
/Users/macdenico/PycharmProjects/PMM-FALP2024/Data Preprocessing/YOLOX


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Using pip 24.2 from /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pip (python 3.12)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Obtaining file:///Users/macdenico/PycharmProjects/PMM-FALP2024/Data%20Preprocessing/YOLOX
  Running command python setup.py egg_info
  /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/setuptools/__init__.py:88: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
  !!

          ********************************************************************************
          Requirements should be satisfied by a PEP 517 installer.
          If you are using pip, you can try `pip install --use-pep517`.
          ********************************************************************************

  !!
    dist.fetch_build_eggs(dist.setup_requires)
  running egg_info
  creating /private/var/folders/ds/4z7lc9ld4mqb377_3t2v4lxm0000gn/T/pip-pip-egg-info-o8n

In [16]:
#Crear las anotaciones de las imágenes

In [10]:
import os
import json
import pandas as pd
import cv2

# Rutas a los directorios y archivos
csv_path = '/Volumes/m2/Memoria/Datasets/Vincomprimido/metacomprimida.csv'  # Ajusta esta ruta
dataset_path = '/Volumes/m2/Memoria/Datasets/Vincomprimidopng'  # Directorio con las imágenes PNG
coco_output = 'annotations_coco.json'

# Cargar el CSV
annotations_df = pd.read_csv(csv_path)

# Inicialización de las estructuras COCO
coco_data = {
    "images": [],
    "annotations": [],
    "categories": []
}

# Definir las categorías (esto depende de tu CSV)
categories = annotations_df['finding_categories'].unique()
category_mapping = {category: i for i, category in enumerate(categories)}

# Añadir categorías al archivo COCO
for category, category_id in category_mapping.items():
    coco_data["categories"].append({
        "id": category_id,
        "name": str(category),
        "supercategory": "none"
    })

annotation_id = 1

# Recorrer cada fila en el CSV
for idx, row in annotations_df.iterrows():
    series_id = row['study_id_x']
    image_id = row['image_id']
    class_id = category_mapping[row['finding_categories']]  # Mapeo de categorías
    xmin, xmax, ymin, ymax = row['xmin'], row['xmax'], row['ymin'], row['ymax']
    
    # Ruta a la imagen PNG
    png_filename = f'{image_id}.png'
    png_path = os.path.join(dataset_path, series_id, png_filename)

    if os.path.exists(png_path):
        # Cargar la imagen para obtener sus dimensiones
        image = cv2.imread(png_path)
        height, width, _ = image.shape

        # Añadir la imagen al archivo COCO
        coco_data["images"].append({
            "id": annotation_id,  # Puedes usar `image_id` si es un número entero único
            "width": width,
            "height": height,
            "file_name": png_filename
        })

        # Convertir las coordenadas de DICOM a COCO
        bbox_width = xmax - xmin
        bbox_height = ymax - ymin

        # Añadir la anotación al archivo COCO
        coco_data["annotations"].append({
            "id": annotation_id,
            "image_id": annotation_id,  # Debe coincidir con el ID de la imagen
            "category_id": class_id,
            "bbox": [xmin, ymin, bbox_width, bbox_height],
            "area": bbox_width * bbox_height,
            "iscrowd": 0
        })

        annotation_id += 1
    else:
        print(f"Imagen no encontrada: {png_path}")

# Guardar las anotaciones en formato COCO
with open(coco_output, 'w') as f:
    json.dump(coco_data, f, indent=4)

print(f"Anotaciones convertidas a formato COCO y guardadas en {coco_output}")


Anotaciones convertidas a formato COCO y guardadas en annotations_coco.json


In [17]:
#Juntar todas las imágenes en una sola carpeta

In [14]:
import os
import shutil

# Rutas a los directorios y archivos
base_path = '/Volumes/m2/Memoria/Datasets/Vincomprimidopng'  # Directorio base donde están las subcarpetas
output_base_path = './dataset'  # Directorio base para el nuevo dataset

# Crear las carpetas de salida si no existen
images_output_path = os.path.join(output_base_path, 'images')


if not os.path.exists(images_output_path):
    os.makedirs(images_output_path)



# Recorrer las subcarpetas dentro de Vincomprimido
for study_id_x in os.listdir(base_path):
    study_path = os.path.join(base_path, study_id_x)
    
    if os.path.isdir(study_path):
        # Recorrer cada imagen en la subcarpeta
        for image_file in os.listdir(study_path):
            if image_file.endswith('.png'):
                # Construir la ruta completa de la imagen original
                image_path = os.path.join(study_path, image_file)
                
                # Crear el nombre de archivo único para la nueva ubicación
                new_image_filename = f'{image_file}'
                new_image_path = os.path.join(images_output_path, new_image_filename)
                
                # Copiar la imagen a la nueva ubicación
                shutil.copy2(image_path, new_image_path)
                print(f'Imagen copiada: {new_image_filename}')




Imagen copiada: 9c0f7d6f89339e7e584a0cf7cbb05038.png
Imagen copiada: 0218fff4e564d49dc159fa98db411c50.png
Imagen copiada: a56874d2c876efaeae69f462d7ac6d05.png
Imagen copiada: 603b45d807c30f1d8687004ca8b5e77c.png
Imagen copiada: 84af32031f2e5b4f442a05433b45e934.png
Imagen copiada: d514ee5a85fe91ab40b1570068929752.png
Imagen copiada: 44a9f36aa7f887ff19e2e266eef45773.png
Imagen copiada: 558d3fb789e3d8fa34f8932d97d9c39e.png
Imagen copiada: 30e17f4eb563d6e3de0fd0d1e99fdab4.png
Imagen copiada: cf435c6f9ddcef01aed9dd1fbdfb98fb.png
Imagen copiada: 3684ab27ba93c1008bd5cffe1c5a0a17.png
Imagen copiada: 931c2664b304fe052723407d6bbf627a.png
Imagen copiada: 0e549350f47b5ca19289c48335e8ebd3.png
Imagen copiada: d2e8fc9197bab88502e2d23bcc5be056.png
Imagen copiada: af0be3ef8316c4ef71c3c9963f9490cb.png
Imagen copiada: 37e100c4d0ff8656539afe81f575e0fa.png
Imagen copiada: 299b526b6c6db3f31338bc961068b0f0.png
Imagen copiada: 266faa805e6a022bc00f11dc89273d25.png
Imagen copiada: ede61a13a41a00251db6fdee5d14ce

In [ ]:
#contar las imágenes y asegurarse que haya 400

In [15]:
import os

# Ruta al directorio base donde están las imágenes PNG
png_base_path = '/Volumes/m2/Memoria/Datasets/dataset'

# Inicializar contador de imágenes
image_count = 0

# Recorrer cada subcarpeta
for series_id in os.listdir(png_base_path):
    series_path = os.path.join(png_base_path, series_id)
    
    if os.path.isdir(series_path):
        # Contar el número de archivos PNG en cada subcarpeta
        for file_name in os.listdir(series_path):
            if file_name.endswith('.png'):
                image_count += 1

print(f'Número total de imágenes en el dataset: {image_count}')


Número total de imágenes en el dataset: 400


In [18]:
!pip install -v -e .
!pip install torch torchvision cython
!pip install -U opencv-python-headless
!pip install -U albumentations
!pip install -U tensorboard


Using pip 24.2 from /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pip (python 3.12)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Obtaining file:///Users/macdenico/PycharmProjects/PMM-FALP2024/Data%20Preprocessing/YOLOX
  Running command python setup.py egg_info
  /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/setuptools/__init__.py:88: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
  !!

          ********************************************************************************
          Requirements should be satisfied by a PEP 517 installer.
          If you are using pip, you can try `pip install --use-pep517`.
          ********************************************************************************

  !!
    dist.fetch_build_eggs(dist.setup_requires)
  running egg_info
  creating /private/var/folders/ds/4z7lc9ld4mqb377_3t2v4lxm0000gn/T/pip-pip-egg-info-p9h

In [19]:
!cp exps/default/yolox_nano.py exps/default/vincomprimido_yolox_nano.py
